for the course "<a target="_blank" href="https://www.udemy.com/course/python-3-deep-dive-part-1/">Python 3: Deep Drive (part 1 - Functional)</a>",<br>
section 7: "Scopes, Closures and Decorators",<br>
(Q&A for <a target="_blank" href="https://www.udemy.com/course/python-3-deep-dive-part-1/learn/lecture/9376820#questions/8320248">lecture 116</a>)

<h3>Decorators</h3>

In [1]:
# simplest decorator
def counter(fn):
    count = 0
    def inner(*args, **kwargs):
        """ This is the inner function """
        nonlocal count
        count += 1
        print(f"Function '{fn.__name__}' has been called {count} times")
        return fn(*args, **kwargs)
    return inner


@counter                                 # add = counter(add)
def add(a:int, b:int=0):
    """ Add two values """
    return a + b

In [2]:
print(add(10,20))

Function 'add' has been called 1 times
30


In [3]:
print(add(30,40))

Function 'add' has been called 2 times
70


In [4]:
# But without 'wrapper' such function has flaws in documentation
print(add.__name__)

inner


In [5]:
help(add)

Help on function inner in module __main__:

inner(*args, **kwargs)
    This is the inner function



In [6]:
print(add.__doc__)

 This is the inner function 


<br>
<hr>

In [7]:
# Using wrapper for more correct documentation
from functools import wraps

def counter(fn):
    count = 0
    @wraps(fn)                           # inner = wraps(fn)(inner)
    def inner(*args, **kwargs):
        """ This is the inner function """
        nonlocal count
        count += 1
        print(f"Function '{fn.__name__}' has been called {count} times")
        return fn(*args, **kwargs)
    return inner

@counter                                 # mult = counter(add)
def mult(a:int, b:int=1):
    """ Multiply two values """
    return a + b

In [8]:
print(mult.__name__)

mult


In [9]:
help(mult)

Help on function mult in module __main__:

mult(a: int, b: int = 1)
    Multiply two values



In [10]:
print(mult.__doc__)

 Multiply two values 


<br>
<hr>

In [11]:
# Memoization of recursive function with limited cache (via special decorator of Python)

from functools import lru_cache   # lru - abbr. from 'least recently used'

@lru_cache(maxsize=8)  # by default 128, more efficient to use the power of 2, None - unlimited cache
def fib(n):
    print(f'calculating fib({n})')
    return 1 if n<=2 else fib(n-1) + fib(n-2)


fib(12)                # all values calculated

calculating fib(12)
calculating fib(11)
calculating fib(10)
calculating fib(9)
calculating fib(8)
calculating fib(7)
calculating fib(6)
calculating fib(5)
calculating fib(4)
calculating fib(3)
calculating fib(2)
calculating fib(1)


144

In [12]:
fib(5)                 # value is taken from cache

5

In [13]:
fib(4)                 # all values calculated again

calculating fib(4)
calculating fib(3)
calculating fib(2)
calculating fib(1)


3

<br>
<br>
<hr>
<h3>Decorator applications</h3>
<br />
Application 1 (timing)

In [16]:
# Simple wrapper, with 1 iteration of the tested function
def timed(fn):
    from time import perf_counter
    from functools import wraps
    
    @wraps(fn)
    def inner(*args, **kwargs):
        start = perf_counter()
        result = fn(*args, **kwargs)
        end = perf_counter()
        elapsed = end - start
        
        args_ = [str(a) for a in args]
        kwargs_ = [f'{k}={v}' for k,v in kwargs.items()]
        all_args = args_ + kwargs_
        args_str = ', '.join(all_args)
        
        print(f'{fn.__name__}({args_str}) took {elapsed:.6f}s to run.')
        
        return result
    
    return inner

In [17]:
# Finding n-th Fibonacci number via recursion

def calc_recursive_fib(n):
    if n<=2:
        return 1
    else:
        return calc_recursive_fib(n-1) + calc_recursive_fib(n-2)

@timed
def fib_recursive(n):
    return calc_recursive_fib(n)


fib_recursive(36)

fib_recursive(36) took 2.221604s to run.


14930352

<br>

In [18]:
# Finding n-th Fibonacci number via loop

@timed
def fib_loop(n):
    fib_1 = 1
    fib_2 = 1
    for i in range(n-2):
        fib_1, fib_2 = fib_2, fib_1+fib_2
        
    return fib_2


fib_loop(36)

fib_loop(36) took 0.000003s to run.


14930352

In [19]:
# Finding n-th Fibonacci number via reduce

from functools import reduce

@timed
def fib_reduce(n):
    initial = (1, 1)
    dummy = range(n-2)
    fib_n = reduce(lambda fs, n: (fs[1], fs[0] + fs[1]), dummy, initial)  # fs - fibonacci sequence
    return fib_n[1]


fib_reduce(36)

fib_reduce(36) took 0.000021s to run.


14930352

<br>
<br>
<hr>
Application 2 (Logger, Stacked decorators)

In [20]:
def logged(fn):
    from functools import wraps
    from datetime import datetime, timezone
    
    @wraps(fn)
    def inner(*args, **kwargs):
        run_dt = datetime.now(timezone.utc)
        result = fn(*args, **kwargs)
        print(f'{run_dt}: called {fn.__name__}')
        return result
    
    return inner


@logged
def func_1():
    pass      

In [21]:
func_1()

2019-10-28 05:45:51.374193+00:00: called func_1


<br>

In [22]:
# Simple wrapper, with 1 iteration of the tested function (copy from above)
def timed(fn):
    from time import perf_counter
    from functools import wraps
    
    @wraps(fn)
    def inner(*args, **kwargs):
        start = perf_counter()
        result = fn(*args, **kwargs)
        end = perf_counter()
        elapsed = end - start
        
        args_ = [str(a) for a in args]
        kwargs_ = [f'{k}={v}' for k,v in kwargs.items()]
        all_args = args_ + kwargs_
        args_str = ', '.join(all_args)
        
        print(f'{fn.__name__}({args_str}) took {elapsed:.6f}s to run.')
        
        return result
    
    return inner

In [23]:
@logged                                  # Stacked decorators:
@timed                                   # fact = logged(timed(fact))
def fact(n):
    from operator import mul             # <=> lambda x,y: x*y
    from functools import reduce
    return reduce(mul, range(1,n+1), 1)

In [24]:
print(fact(6))

fact(6) took 0.000018s to run.
2019-10-28 05:52:28.604572+00:00: called fact
720


<br>

In [34]:
# Small researches, in what order decorators and functions work

def logged(fn):
    print('logged decorator running...')
    @timed
    def inner():
        print('logged inner running...')
        fn()
    return inner

def timed(fn):
    print('timer decorator running...')
    def inner():
        print('timer inner running...')
        fn()
    return inner

@logged
def fact():
    print('fact running...')

fact()

logged decorator running...
timer decorator running...
timer inner running...
logged inner running...
fact running...


In [35]:
def logged(fn):
    print('logged decorator running...')
    @timed
    def inner_logged():
        print('inner_logged running...')
        fn()
    return inner_logged

def timed(gn):
    print('timer decorator running...')
    def inner_timed():
        print('inner_timed running...')
        gn()
    return inner_timed

@logged
def fact():
    print('fact running...')
    
fact()

logged decorator running...
timer decorator running...
inner_timed running...
inner_logged running...
fact running...


In [36]:
def logged(fn):
    print('logged decorator running...')
    @timed
    def logged_inner():
        print('logged_inner running...')
        fn()
    return logged_inner

def timed(gn):
    print('timer decorator running...')
    def timed_inner():
        print('timed_inner running...\n')
        gn()
    return timed_inner

@logged
def fact():
    print('fact running...')
    
fact()

logged decorator running...
timer decorator running...
timed_inner running...

logged_inner running...
fact running...


<br>
<hr>
Futher experiments with calculation of Fibonacci number

In [25]:
# Simple wrapper, with 1 iteration of the tested function (copy from above)
def timed(fn):
    from time import perf_counter
    from functools import wraps
    
    @wraps(fn)
    def inner(*args, **kwargs):
        start = perf_counter()
        result = fn(*args, **kwargs)
        end = perf_counter()
        elapsed = end - start
        
        args_ = [str(a) for a in args]
        kwargs_ = [f'{k}={v}' for k,v in kwargs.items()]
        all_args = args_ + kwargs_
        args_str = ', '.join(all_args)
        
        print(f'{fn.__name__}({args_str}) took {elapsed:.6f}s to run.')
        
        return result
    
    return inner

In [26]:
# Finding n-th Fibonacci number via recursion with cashe (my version)

@timed
def calc_recursive_fib_cache(n):
    fib_cache = {1:1, 2:1}
    
    def inner(n):
        if n not in fib_cache:
            fib_cache[n] = inner(n-1) + inner(n-2)
        return fib_cache[n]
    return inner(n)


print(calc_recursive_fib_cache(36))

calc_recursive_fib_cache(36) took 0.000017s to run.
14930352


In [27]:
# instructor's version
@timed
def calc_recursive_fib_cache():
    fib_cache = {1:1, 2:1}
    
    def inner(n):
        if n not in fib_cache:
            fib_cache[n] = inner(n-1) + inner(n-2)
        return fib_cache[n]
    return inner


f = calc_recursive_fib_cache()
print(f(36))

calc_recursive_fib_cache() took 0.000002s to run.
14930352


In [28]:
# via class
@timed
class Fib:
    def __init__(self):
        self.cache = {1:1, 2:1}
    
    def fib(self, n):
        if n not in self.cache:
            self.cache[n] = self.fib(n-1) + self.fib(n-2)
        return self.cache[n]


f = Fib()
print(f.fib(36))

Fib() took 0.000001s to run.
14930352


In [29]:
# via decorator
@timed
def memoize(fn):
    cache = {}
    
    def inner(n):
        if n not in cache:
            cache[n] = fn(n)
        return cache[n]
    
    return inner

@memoize
def fib(n):
    return 1 if n<=2 else fib(n-1) + fib(n-2)


print(fib(36))
print()

memoize(<function fib at 0x7f63689f1b70>) took 0.000002s to run.
14930352



In [30]:
# Memoization of recursive function with limited cache (via special decorator of Python)

from functools import lru_cache
# lru - abbr. from 'least recently used'

@lru_cache(maxsize=8)  # Def. 128, more efficient to use the power of 2, None - unlimited cache
def fib(n):
    print(f'calculating fib({n})')
    return 1 if n<=2 else fib(n-1) + fib(n-2)


fib(12)

calculating fib(12)
calculating fib(11)
calculating fib(10)
calculating fib(9)
calculating fib(8)
calculating fib(7)
calculating fib(6)
calculating fib(5)
calculating fib(4)
calculating fib(3)
calculating fib(2)
calculating fib(1)


144

In [31]:
fib(5)

5

In [32]:
fib(4)

calculating fib(4)
calculating fib(3)
calculating fib(2)
calculating fib(1)


3

<br>

#### Counter of function calls

In [33]:
def op_count(fn):
    from functools import wraps
    
    count = 0
    
    @wraps(fn)
    def inner(*args, **kwargs):
        nonlocal count  
        result = fn(*args, **kwargs)
        count += 1
        print('number of operations', count)
        return result

    def reset():
        nonlocal count
        print('resetting count...')
        count = 0

    inner.reset = reset
    return inner


@op_count
def f1():
    return '   f1 works'


for _ in range(3):
    print(f1())

f1.reset()

for _ in range(3):
    print(f1())

number of operations 1
   f1 works
number of operations 2
   f1 works
number of operations 3
   f1 works
resetting count...
number of operations 1
   f1 works
number of operations 2
   f1 works
number of operations 3
   f1 works


<br>
<br>
<br>
<hr>
<h3>Decorators with parameter</h3

In [34]:
# Simple wrapper, with 1 iteration of the tested function (copy from above)
def timed(fn):
    from time import perf_counter
    from functools import wraps
    
    @wraps(fn)
    def inner(*args, **kwargs):
        start = perf_counter()
        result = fn(*args, **kwargs)
        end = perf_counter()
        elapsed = end - start
        
        args_ = [str(a) for a in args]
        kwargs_ = [f'{k}={v}' for k,v in kwargs.items()]
        all_args = args_ + kwargs_
        args_str = ', '.join(all_args)
        
        print(f'{fn.__name__}({args_str}) took {elapsed:.6f}s')
        
        return result
    
    return inner

In [35]:
# Creating a decorator factory
def timed_factory(count):
    def timed(fn):
        from time import perf_counter
        from functools import wraps

        @wraps(fn)
        def inner(*args, **kwargs):
            elapsed_total = 0

            for i in range(count):        
                start = perf_counter()
                result = fn(*args, **kwargs)
                end = perf_counter()
                elapsed = end - start
                elapsed_total += elapsed

            args_ = [str(a) for a in args]
            kwargs_ = [f'{k}={v}' for k,v in kwargs.items()]
            all_args = args_ + kwargs_
            args_str = ', '.join(all_args)

            elapsed_avg = elapsed_total / count
            print(f'{fn.__name__}({args_str}) took {elapsed_avg:.6f}s, {count} runs')

            return result

        return inner
    
    return timed

In [36]:
def cal_fib_recursive(n):
    return 1 if n<=2 else cal_fib_recursive(n-1) + cal_fib_recursive(n-2)

@timed_factory(10)
def fib(n):
    return cal_fib_recursive(n)

fib(28)

fib(28) took 0.051844s, 10 runs


317811

<br>
<br>
<br>
<hr>
<h3>Decorators in general form</h3>
<h4>simple decorator</h4>

(Q&A for <a target="_blank" href="https://www.udemy.com/course/python-3-deep-dive-part-1/learn/lecture/9376820#questions/8309596">lecture 116</a>)

In [37]:
def dec(fn):
    print("running dec")
    
    def inner(*args, **kwargs):
        print("running inner")
        return fn(*args, **kwargs)
    
    return inner

In [38]:
@dec                         # f1 = dec(f1)
def f1():
    print("running f1")

running dec


In [39]:
f1()

running inner
running f1


<br>
<h4>decorator factory (decorator with parameter)</h4>

In [40]:
def dec_factory(n):
    print("running dec_factory")
    
    def dec(fn):
        print(f"running dec with parameter n={n}")

        def inner(*args, **kwargs):
            print("running inner")
            return fn(*args, **kwargs)

        return inner

    return dec

In [41]:
@dec_factory(10)             # f2 = dec_factory(10)(f2)
def f2():
    print("running f2")

running dec_factory
running dec with parameter n=10


In [42]:
f2()

running inner
running f2


<br>
<h4>Decorator class</h4>

In [43]:
# Decorator factory for comparison
def my_dec(a, b):
    def dec(fn):
        def inner(*args, **kwargs):
            print(f'decorated function called: a={a}, b={b}')
            return fn(*args, **kwargs)
        return inner
    return dec


@my_dec(10, 20)
def f1(st):
    print(f'Hello {st}')


f1('World')

decorated function called: a=10, b=20
Hello World


In [44]:
# The work of traditional class
class MyClass:
    def __init__(self, a, b):
        self.a = a
        self.b = b
        
    def __call__(self, c):
        print(f'called a={self.a}, b={self.b}, c={c}')


obj = MyClass(10, 20)


obj(100)                  # obj.__call__(100)

called a=10, b=20, c=100


In [45]:
# Decoration via class (example 1)
class MyClass:
    def __init__(self, a, b):
        self.a = a
        self.b = b
        
    def __call__(self, fn):
        def inner(*args, **kwargs):
            print(f'decorated function called: a={self.a}, b={self.b}')
            return fn(*args, **kwargs)
        return inner


@MyClass(30, 40)
def f2(st):
    print(f'Hello {st}')


f2('Python')

decorated function called: a=30, b=40
Hello Python


In [46]:
# Decoration via class (example 2, in general form)
class DecClass:
    def __init__(self, n):
        print('running DecClass')
        self.n = n
    
    def __call__(self, fn):
        print(f'running __call__ with parameter n={self.n}')
        def inner(*args, **kwargs):
            print('running inner')
            return fn(*args, **kwargs)
        return inner

In [47]:
@DecClass(10)
def f3():
    print('running f3')  

running DecClass
running __call__ with parameter n=10


In [48]:
f3()

running inner
running f3


<br>

<h4>Decorating classes</h4>

In [128]:
# simplest example
class Crow:
    pass

def speak(cls):
    cls.speak = lambda self, msg: f'{self.__class__.__name__} says: {msg}'  # "monkey patching"
    return cls
    
Crow = speak(Crow)


cr = Crow()
print(cr.speak('Kah'))

Crow says: Kah


<br>

In [49]:
# more complicated examle:
# extracting infomation about an object via applying decorator to a class
def info(self):
    results = []
    results.append(f'class: {self.__class__.__name__}')
    for k,v in vars(self).items():
        results.append(f'{k}: {v}')
    return results

def debug_info(cls):
    cls.debug = info
    return cls  # we return the class only for using decorator syntax

@debug_info             # Person = debug_info(Person)
class Person:
    def __init__(self, name, birth_year):
        self.name = name
        self.birth_year = birth_year


p = Person('Xena', 1995)

p.debug()

['class: Person', 'name: Xena', 'birth_year: 1995']

<br>

<br>

<br>

### Dispatching (Decorator Application)

Dispatch is the act of sending something somewhere.

#### I. Our implementation

In [1]:
from html import escape

In [2]:
def singledispatch(fn):
    registry = {object: fn}   # initial dictionary
    
    def decorated(arg):
        """ Choose applied function according type of given argument """
        f = registry.get(type(arg), registry[object])
        return f(arg)
        
    def register(type_):
        """ Register new couple 'type: function' """
        def inner(fn):
            registry[type_] = fn
            return fn              # we need this return for stacking register recorators
        return inner

    def dispatch(type_):
        """ Allow user to check what function is used for given type """
        return registry.get(type_, registry[object])
    
    decorated.register = register
    decorated.registry = registry  # for security it's better don't open access,
                                   # only for debugging purpose
    decorated.dispatch = dispatch
        
    return decorated

In [3]:
# Set default couple 'object: function'
@singledispatch
def htmlize(a):             # htmlize now our default function, for object
    return escape(str(a))

In [4]:
htmlize.registry            # check content of inner dictionary 'registry'

{object: <function __main__.htmlize(a)>}

In [5]:
# register new couple 'type: function'
@htmlize.register(int)     # html_int = htmlize.register(int)(html_int)
def html_int(a):
    return f'{a}(<i>({hex(a)})</i>)'

In [6]:
htmlize.registry            # check content of inner dictionary 'registry' again

{object: <function __main__.htmlize(a)>, int: <function __main__.html_int(a)>}

In [7]:
htmlize.dispatch(int)       # check what function is used for type 'int'

<function __main__.html_int(a)>

In [8]:
print(htmlize('0 < 1'))     # check of work 1

0 &lt; 1


In [9]:
print(htmlize(255))         # check of work 2

255(<i>(0xff)</i>)


In [10]:
# register simultaneously two couple 'type1: function1', 'type2: function1'
@htmlize.register(tuple)  # html_sequence = htmlize.register(tuple)(html_sequence)
@htmlize.register(list)   # html_sequence = htmlize.register(list)(html_sequence)
def html_sequence(ls):
    items = (f'<li>{htmlize(item)}</li>' for item in ls)
    return '<ul>\n' + '\n'.join(items) + '\n</ul>'

In [11]:
htmlize.registry            # check content of inner dictionary 'registry'

{object: <function __main__.htmlize(a)>,
 int: <function __main__.html_int(a)>,
 list: <function __main__.html_sequence(ls)>,
 tuple: <function __main__.html_sequence(ls)>}

In [12]:
print(htmlize([10, 20, 30]))  # check of work 3

<ul>
<li>10(<i>(0xa)</i>)</li>
<li>20(<i>(0x14)</i>)</li>
<li>30(<i>(0x1e)</i>)</li>
</ul>


Our implementation is simplified and there're problems if, e.g., we try to use Integral and Sequence types.

<br>

<br>

#### II. Implementation using module functools

In [1]:
from functools import singledispatch
from numbers import Integral
from collections.abc import Sequence
from html import escape

In [2]:
# Set default couple 'object: function'
@singledispatch
def htmlize(a):             # htmlize now our default function, for object
    return escape(str(a))

In [3]:
htmlize.registry            # check content of inner dictionary 'registry'

mappingproxy({object: <function __main__.htmlize(a)>})

In [4]:
# register new couple 'type: function'
@htmlize.register(Integral)
def htmlize_integral_number(a):
    return f'{a}(<i>({hex(a)})</i>)'

In [5]:
htmlize.registry            # check content of inner dictionary 'registry' again

mappingproxy({object: <function __main__.htmlize(a)>,
              numbers.Integral: <function __main__.htmlize_integral_number(a)>})

In [6]:
htmlize.dispatch(int)       # check what function is used for type 'int'

<function __main__.htmlize_integral_number(a)>

In [7]:
print(htmlize('0 < 1'))     # check of work 1

0 &lt; 1


In [8]:
print(htmlize(255))         # check of work 2

255(<i>(0xff)</i>)


In [9]:
@htmlize.register(Sequence)  # html_sequence = htmlize.register(Sequence)(html_sequence)
def html_sequence(ls):
    items = (f'<li>{htmlize(item)}</li>' for item in ls)
    return '<ul>\n' + '\n'.join(items) + '\n</ul>'

In [10]:
htmlize.registry            # check content of inner dictionary 'registry'

mappingproxy({object: <function __main__.htmlize(a)>,
              numbers.Integral: <function __main__.htmlize_integral_number(a)>,
              collections.abc.Sequence: <function __main__.html_sequence(ls)>})

In [11]:
print(htmlize([10, 20, 30]))  # check of work 3

<ul>
<li>10(<i>(0xa)</i>)</li>
<li>20(<i>(0x14)</i>)</li>
<li>30(<i>(0x1e)</i>)</li>
</ul>


<br>

Error then sequence is applied to string:

In [12]:
print(htmlize('python'))

RecursionError: maximum recursion depth exceeded

In [13]:
# Correction of aforementioned error 
@htmlize.register(str)
def html_str(s):
    return escape(str(s)).replace('\n', '<br/>\n')

In [14]:
print(htmlize('python'))   # check of work 4

python
